In [8]:
import pandas as pd
import cohere
df = pd.read_csv('/Users/saikrishna/Downloads/GetGitData/datab.csv')

In [9]:
language_columns = [col for col in df.columns if '%' in col]
df['Skills'] = df.apply(lambda row: {lang.replace('%', ''): row[lang] for lang in language_columns if row[lang] > 0}, axis=1)

In [10]:
def generate_summary(row):
    skills = row['Skills']
    sorted_skills = sorted(skills.items(), key=lambda x: x[1], reverse=True)
    skill_descriptions = [f"{skill} ({percentage}%)" for skill, percentage in sorted_skills]
    return f"User {row['Username']} is proficient in {', '.join(skill_descriptions)}."
df['SkillSummary'] = df.apply(generate_summary, axis=1)


In [11]:
print(df)

          Username             CreatedAt  \
0          savolai  2011-06-11T18:04:13Z   
1         jantatje  2014-04-21T09:39:45Z   
2         dharnnie  2016-11-29T10:15:26Z   
3           zengsn  2011-02-20T05:48:40Z   
4    shriroopjoshi  2013-08-11T11:06:26Z   
..             ...                   ...   
642      sonicning  2013-12-19T05:00:12Z   
643  shubhamsingh5  2016-09-19T21:11:32Z   
644     GramGibson  2010-06-29T20:22:11Z   
645    minecrawler  2011-10-15T23:34:28Z   
646  elementalvoid  2009-11-20T23:03:24Z   

                                             AvatarUrl  Id  Contributions  \
0    https://avatars.githubusercontent.com/u/844183...   1             90   
1    https://avatars.githubusercontent.com/u/736027...   1              8   
2    https://avatars.githubusercontent.com/u/242459...   2              0   
3    https://avatars.githubusercontent.com/u/627946...   1            659   
4    https://avatars.githubusercontent.com/u/520712...   1              1   
..       

In [12]:
import cohere
co = cohere.Client('eEnhJhgFPTaGPmZiFzIwWVmZSHADwdPDlLECeZDe')


In [16]:
def identify_weakness(summary):
    prompt = f"{summary}\n\nIdentify the user's weaknesses and areas for improvement."
    response = co.generate(model='command', prompt=prompt, max_tokens=50)
    return response.generations[0].text.strip()
df['Weaknesses'] = df['SkillSummary'].apply(identify_weakness)

In [20]:
embeddings = co.embed(texts=df['SkillSummary'].tolist(), model='large').embeddings
df['Embeddings'] = embeddings

In [23]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

similarity_matrix = cosine_similarity(embeddings)


In [24]:
def find_complementary_users(user_index):
    user_similarities = similarity_matrix[user_index]
    complementary_indices = np.argsort(user_similarities)
    # Exclude the user themselves
    complementary_indices = complementary_indices[complementary_indices != user_index]
    return df.iloc[complementary_indices[:5]]['Username'].tolist()
df['ComplementaryUsers'] = df.index.map(find_complementary_users)

In [25]:
def generate_recommendation(row):
    recommendation = f"User {row['Username']} could improve in {row['Weaknesses']}.\n"
    recommendation += f"Potential teammates to complement their skills: {', '.join(row['ComplementaryUsers'])}."
    return recommendation
df['Recommendation'] = df.apply(generate_recommendation, axis=1)


In [26]:
df[['Username', 'Recommendation']].to_csv('recommendations.csv', index=False)
